In [41]:
from lusidtools.jupyter_tools import toggle_code

"""FX Option - Valuation Workflow

Attributes
----------
FX Options
complex market data
recipes
valuations
black scholes
"""

toggle_code("Toggle Docstring")

# Booking and valuing an FX Option

In this notebook, we demonstrate how an FX Option can be booked in LUSID, and extend it to use cases such as running a standard valuation and calculating PnL.

Table of Contents:

In [42]:
# Import generic non-LUSID packages
import os
import pandas as pd
from datetime import datetime, timedelta
import json
import pytz
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as lm
import lusid.api as la
from lusid.utilities import ApiClientFactory
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import key functions from Lusid-Python-Tools and other packages
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.4f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path)

print('LUSID Environment Initialised')
print('LUSID API Version :', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version : 0.6.8245.0


In [43]:
#Set required APIs
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
complex_market_data_api = api_factory.build(lusid.api.ComplexMarketDataApi)
aggregration_api = api_factory.build(lusid.api.AggregationApi)

In [44]:
# Define scopes
scope = "ibor"
market_data_scope = "ibor"

# 1. Create Portfolio
We begin by creating a portfolio that will contain the FX forward instrument that we will be looking to price.

In [45]:
portfolio_code = "FxOptionIntrinsic"

try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])


# 2. Create Instrument
Using the below method we can reference the SDK models to populate the required parameters for creating an FX Option. We then populate the instrument variables, and make a call to persist the instrument in LUSID.

In [46]:
# Define a function that creates an FX option
def create_fx_option(
        name,
        start_date,
        option_maturity_date,
        option_settlement_date,
        is_delivery_not_cash,
        is_call_not_put,
        strike,
        dom_ccy,
        fgn_ccy,
        premium=None,
        premium_ccy=None,
        premium_date=None
):
    if premium:
        prem_obj = lm.Premium(
            amount=premium,
            currency=premium_ccy if premium_ccy else dom_ccy,
            date=premium_date if premium_date else option_maturity_date
        )
    else:
        prem_obj = None

    fx_option = lm.FxOption(
        start_date=start_date,
        option_maturity_date=option_maturity_date,
        option_settlement_date=option_settlement_date,
        is_delivery_not_cash=is_delivery_not_cash,
        is_call_not_put=is_call_not_put,
        strike=strike,
        dom_ccy=dom_ccy,
        fgn_ccy=fgn_ccy,
        premium=prem_obj,
        instrument_type="FxOption"
    )

    return fx_option


# Define a function to upsert instrument
def upsert_instrument_to_lusid(instrument_definition, name, identifier, identifier_type):
    return instruments_api.upsert_instruments(
        request_body={
            identifier: lm.InstrumentDefinition(
                name=name,
                identifiers={
                    identifier_type: lm.InstrumentIdValue(value=identifier)
                },
                definition=instrument_definition,
            )
        }
    )

For this example we will be creating a 6M EUR/USD forward, with the following characteristics:

Forward Exchange Rate (Strike): 1.21552025
Buy/Receive: EUR
Sell/Pay: USD
Start Date: 20 Jan 2021
Maturity Date: 20 July 2021

In [47]:
# Set the instrument variables
option_name = "EUR/USD 6M FX Option 20210720"
option_identifier = "OPT-EURUSD20210720"
identifier_type = "ClientInternal"
description = "EUR/USD FX Option 20210720"
start_date = datetime(2021, 1, 20, tzinfo=pytz.utc)
maturity_date = datetime(2021, 7, 20, tzinfo=pytz.utc)
settlement_days = 2
dom_ccy = "EUR"
fgn_ccy = "USD"
strike = 1.15
is_call_not_put = True
is_delivery_not_cash = False

In [48]:
opt_definition = create_fx_option(
    option_name,
    start_date,
    maturity_date,
    maturity_date + timedelta(settlement_days),
    is_delivery_not_cash,
    is_call_not_put,
    strike,
    dom_ccy,
    fgn_ccy,
)

# Upsert the instrument
upsert_response = upsert_instrument_to_lusid(opt_definition, option_name, option_identifier, identifier_type)
luid = upsert_response.values[option_identifier].lusid_instrument_id
print(luid)

LUID_0000HKQZ


# 3. Upsert transactions
Having created the portfolio, we add a StockIn transaction against the forward to create a position without incurring any costs. Notice that this transaction type will simply increase our position by the given units, as opposed to a typical Buy/Sell, that would also impact cash.

In [49]:
# Set trade variables
trade_date = datetime(2021, 1, 20, tzinfo=pytz.utc)
settle_days = 2
units = 1000000

# Book a StockIn transaction against the forward
fwd_txn = lm.TransactionRequest(
    transaction_id="TXN001",
    type="StockIn",
    instrument_identifiers={"Instrument/default/ClientInternal": option_identifier},
    transaction_date=trade_date,
    settlement_date=trade_date + timedelta(days=settle_days),
    units=units,
    transaction_price=lm.TransactionPrice(price=1,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=1,currency="USD"),
    exchange_rate=1,
    transaction_currency="USD"
)

response = transaction_portfolios_api.upsert_transactions(scope=scope,
                                                    code=portfolio_code,
                                                    transaction_request=[fwd_txn])

print(f"Transaction succesfully updated at time: {response.version.as_at_date}")

Transaction succesfully updated at time: 2021-11-30 10:59:19.456753+00:00


# 4. Upsert FX Market Data


# 4.1 FX Spot Rates
We begin by adding the FX spot rates, which are the first piece of market data required to value an FX forward on a given day.

In [50]:
# Read fx spot rates and make datetimes timezone aware
quotes_df = pd.read_csv("data/eurusd_spot.csv")
quotes_df["Date"] = pd.to_datetime(quotes_df["Date"], dayfirst=True)
quotes_df["Date"] = quotes_df["Date"].apply(lambda x: x.replace(tzinfo=pytz.utc))
quotes_df.head()

,Date,Rate,Pair
0,2021-01-01 00:00:00+00:00,1.2215,EUR/USD
1,2021-01-04 00:00:00+00:00,1.2248,EUR/USD
2,2021-01-05 00:00:00+00:00,1.2298,EUR/USD
3,2021-01-06 00:00:00+00:00,1.2327,EUR/USD
4,2021-01-07 00:00:00+00:00,1.2272,EUR/USD


In [51]:
# Create quotes request
instrument_quotes = {
            index: lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=row["Pair"],
                    instrument_id_type="CurrencyPair",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=row["Date"],
            ),
            metric_value=lm.MetricValue(value=row["Rate"], unit=row["Pair"]),
        )
    for index, row in quotes_df.iterrows()
}

# Upsert quotes into LUSID
response = quotes_api.upsert_quotes(
    scope=market_data_scope, request_body=instrument_quotes
)

if response.failed == {}:
    print(f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded.")
else:
    print(f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quotes successfully loaded into LUSID. 223 quotes loaded.


In [52]:
# Set recipe code
recipe_code = "FxOptIntrinsic"

# Populate recipe parameters
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code=recipe_code,
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Fx.CurrencyPair.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            )
        ],
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="ConstantTimeValueOfMoney",
                instrument_type="FxOption",
                parameters="{}",
            )
        ],
    ),
)

response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )


print(f"Configuration recipe loaded into LUSID at time {response.value}.")

Configuration recipe loaded into LUSID at time 2021-11-30 10:59:20.705456+00:00.


In [53]:
# Set the queryable metrics and operation to be applied on the group_by level
metrics = [
    lm.AggregateSpec("Instrument/default/Name", "Value"),
    lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
    lm.AggregateSpec("Valuation/PvInPortfolioCcy", "Value"),
    lm.AggregateSpec("Valuation/Diagnostics/FxForwardRate", "Value"),
    lm.AggregateSpec("Quotes/FxRate/DomFgn", "Value"),
    lm.AggregateSpec("Holding/default/Units", "Value"),
    lm.AggregateSpec("Valuation/PnL/Tm1/Pfolio", "Value"),
        ]

# Method to query daily valuations
def get_daily_valuation(date, portfolio_code, recipe_code, metrics, group_by=["Instrument/default/Name"]):

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code=recipe_code),
        metrics=metrics,
        group_by=group_by,
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Valuation/PvInPortfolioCcy": "Market Value",
            "Valuation/Diagnostics/FxForwardRate": "Forward Rate (Interpolated)",
            "Quotes/FxRate/DomFgn": "FX Spot Rate",
            "Valuation/PnL/Tm1/Pfolio": "PnL (1-day)"
        },
        inplace=True,
    )

    return vals_df

In [54]:
df = get_daily_valuation(trade_date, portfolio_code, recipe_code, metrics)
#df.drop('Aggregation/Errors', axis=1, inplace=True)
df

,InstrumentName,ClientInternal,Market Value,Forward Rate (Interpolated),FX Spot Rate,Holding/default/Units,PnL (1-day)
0,EUR/USD 6M FX Option 20210720,OPT-EURUSD20210720,"60,600.0000",None,1.2106,"1,000,000.0000","-2,180.7239"


In [55]:
df = get_daily_valuation(trade_date + timedelta(days=1), portfolio_code, recipe_code, metrics)
#df.drop('Aggregation/Errors', axis=1, inplace=True)
df

,InstrumentName,ClientInternal,Market Value,Forward Rate (Interpolated),FX Spot Rate,Holding/default/Units,PnL (1-day)
0,EUR/USD 6M FX Option 20210720,OPT-EURUSD20210720,"66,400.0000",None,1.2164,"1,000,000.0000","5,509.6646"


In [56]:
df = get_daily_valuation(trade_date + timedelta(days=2), portfolio_code, recipe_code, metrics)
#df.drop('Aggregation/Errors', axis=1, inplace=True)
df

,InstrumentName,ClientInternal,Market Value,Forward Rate (Interpolated),FX Spot Rate,Holding/default/Units,PnL (1-day)
0,EUR/USD 6M FX Option 20210720,OPT-EURUSD20210720,"67,100.0000",None,1.2171,"1,000,000.0000",661.7889


In [57]:

df = get_daily_valuation(maturity_date - timedelta(days=1), portfolio_code, recipe_code, metrics)
#df.drop('Aggregation/Errors', axis=1, inplace=True)
df

,InstrumentName,ClientInternal,Market Value,Forward Rate (Interpolated),FX Spot Rate,Holding/default/Units,PnL (1-day)
0,EUR/USD 6M FX Option 20210720,OPT-EURUSD20210720,"30,000.0000",None,1.1800,"1,000,000.0000",-584.4486


In [58]:
upsertable_cash_flows = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
    scope=scope,
    code=portfolio_code,
    recipe_id_code=recipe_code,
    recipe_id_scope=scope,
    effective_at=maturity_date,
    window_start=trade_date,
    window_end=maturity_date,
)

lusid_response_to_data_frame(upsertable_cash_flows)

ApiException: (500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 30 Nov 2021 10:59:24 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Rate-Limit-Limit': '1m', 'X-Rate-Limit-Remaining': '4999', 'X-Rate-Limit-Reset': '2021-11-30T11:00:23.9221387Z', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HMDJTECVT6KV:0000002F', 'lusid-meta-correlationId': '0HMDJTECVT6KV:0000002F', 'lusid-meta-duration': '890', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com; style-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; report-uri https://lusid.report-uri.com/r